In [1]:
import ccxt
import pandas as pd 
from tqdm import tqdm

/Users/leahxue/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def get_data(exchange, s, date): 
    timeframe = '1m'
    bars = 100000
    # since = exchange.parse8601(f'{date}T00:00:00Z')
    since = exchange.parse8601(date)
    ohlcv = exchange.fetch_ohlcv(s, timeframe, since, bars)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'bid', 'ask', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

In [3]:
exchange = ccxt.binanceus()

# extract all symbols supported by the exchange
exchange.load_markets()
symbols = exchange.symbols

In [16]:
timeframe = '1m'
bars = 100000
since = exchange.parse8601(f'2022-01-01T00:00:00Z')

In [21]:
exchange.fetchOrders('BTC/USDT', since, bars)

AuthenticationError: binanceus requires "apiKey" credential

In [4]:
# example: 
dfs = []
for s in symbols[:10]: 
    try: 
        df = get_data(exchange, s, '2021-01-12')
        df["symbol"] = s
        dfs.append(df)
    except: 
        pass

In [5]:
dfs[-1].index[-1].year

2024

Note, I think limit is around 999? so need to get separately and concate together 

In [11]:
def get_data_per_symbol(s): 
    start = "2022-01-01T00:00:00Z"
    dfs = []
    try: 
        df = get_data(exchange, s, start)
    except: 
        return []
    
    if len(df.index) == 0: 
        return []
    last_date = df.index[-1] 
    dfs.append(df)
    while (last_date.year == 2022) and (last_date.month == 1) and (last_date.day <= 2): 
        time = pd.concat(dfs).index[-1]
        time = time.strftime("%Y-%m-%dT%H:%M:%SZ")
        try: 
            df = get_data(exchange, s, time)
        except:
            break
        last_date = df.index[-1]
        dfs.append(df)
    df = pd.concat(dfs)
    df["symbol"] = s
    return df

In [21]:
dfs = []
for s in tqdm(symbols): 
    df = get_data_per_symbol(s) 
    if len(df) == 0: 
        continue
    if len(df) > 0: 
        dfs.append(df)

100%|██████████| 541/541 [00:36<00:00, 14.80it/s]


In [22]:
pd.concat(dfs).to_csv("data.csv")

In [101]:
for s in tqdm(symbols): 
    df = get_data_per_symbol(s) 
    if len(df) == 0: 
        continue
    base = s.split("/")[0]
    quote = s.split("/")[1]
    df.to_csv(f"data/{base}_{quote}.csv")

  5%|▍         | 27/541 [20:18<6:26:43, 45.14s/it]


KeyboardInterrupt: 